In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install open3d

In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

In [4]:
# Define file paths
data_path = '/content/drive/MyDrive/DSC 210 Final Project/Dataset'
output_path = '/content/drive/MyDrive/DSC 210 Final Project/gaussian_splatting_output'
output_features = '/content/drive/MyDrive/DSC 210 Final Project/gaussian_splatting_output_features'
os.makedirs(output_path, exist_ok=True)
os.makedirs(output_features, exist_ok=True)

In [ ]:
# Load images
def load_images(path):
    images = []
    for file_name in sorted(os.listdir(path)):
        if file_name.lower().endswith(('png', 'jpg', 'jpeg')):
            img = cv2.imread(os.path.join(path, file_name))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)
    return images

# Apply Gaussian splatting
def gaussian_splat(image, sigma=5):
    normalized_image = image / 255.0
    # Apply Gaussian filter
    splatted_image = gaussian_filter(normalized_image, sigma=sigma)
    return np.clip(splatted_image * 255, 0, 255).astype(np.uint8)

# Detect keypoints and descriptors (SIFT)
def detect_features(image):
    sift = cv2.SIFT_create()
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return keypoints, descriptors

# Match features between two images
def match_features(kp1, kp2, des1, des2):
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches[:400]

# Draw top n matches on images
def draw_matches(img1, img2, kp1, kp2, matches):
    img1_with_matches = img1.copy()
    img2_with_matches = img2.copy()
    match_img = cv2.drawMatches(
        img1_with_matches, kp1, img2_with_matches, kp2, matches, None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )
    return match_img

# Load images
images = load_images(data_path)

splatted_images = []
for i in range(len(images) - 1):
    img1 = images[i]
    img2 = images[i+1]

    # Detect keypoints and descriptors for both images
    kp1, des1 = detect_features(img1)
    kp2, des2 = detect_features(img2)

    # Match features between the two images
    matches = match_features(kp1, kp2, des1, des2)

    # Draw matches between the two images
    match_img = draw_matches(img1, img2, kp1, kp2, matches)

    # Display matches
    plt.figure(figsize=(10, 5))
    plt.imshow(match_img)
    plt.axis("off")
    plt.title(f"Top 5 Feature Matches between Image {i+1} and Image {i+2}")
    plt.show()

    splatted_img1 = gaussian_splat(img1, sigma=5)
    splatted_img2 = gaussian_splat(img2, sigma=5)

    splatted_images.append(splatted_img1)
    splatted_images.append(splatted_img2)
    output_file1 = os.path.join(output_path, f"splatted_{i+1}.png")
    output_file2 = os.path.join(output_path, f"splatted_{i+2}.png")
    cv2.imwrite(output_file1, cv2.cvtColor(splatted_img1, cv2.COLOR_RGB2BGR))
    cv2.imwrite(output_file2, cv2.cvtColor(splatted_img2, cv2.COLOR_RGB2BGR))

plt.figure(figsize=(15, 5))
for i in range(len(images)):
    # Original Image
    plt.subplot(2, len(images), i+1)
    plt.imshow(images[i])
    plt.axis("off")
    plt.title(f"Original {i+1}")

    # Splatted Image
    plt.subplot(2, len(images), len(images)+i+1)
    plt.imshow(splatted_images[i])
    plt.axis("off")
    plt.title(f"Splatted {i+1}")

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [5]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from sklearn.neighbors import LocalOutlierFactor, NearestNeighbors
import open3d as o3d

class GaussianSplatting:
    def __init__(self, input_path, output_path):
        self.input_path = input_path
        self.output_path = output_path
        os.makedirs(output_path, exist_ok=True)

        # Camera params
        self.K = np.array([
            [2883.79868730, 0, 1500],
            [0, 2883.79868730, 2000],
            [0, 0, 1]
        ])

    def load_images(self):
        images = []
        valid_extensions = ('.jpg')

        if not os.path.exists(self.input_path):
            raise FileNotFoundError(f"Directory {self.input_path} not found")

        files = [f for f in sorted(os.listdir(self.input_path))
                 if f.lower().endswith(valid_extensions)]

        if not files:
            raise ValueError(f"No valid images found in {self.input_path}")

        for file_name in files:
            img = cv2.imread(os.path.join(self.input_path, file_name))
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                images.append(img)

        return images

    def extract_features(self, images):
        sift = cv2.SIFT_create(
            nfeatures=9000,
            nOctaveLayers=5,
            contrastThreshold=0.02,
            edgeThreshold=10,
            sigma=1.2
        )

        features = []
        responses = []

        for i, image in enumerate(images):
            kp, des = sift.detectAndCompute(image, None)

            if kp and des is not None:
                kp_data = {
                    'coords': np.array([k.pt for k in kp]),
                    'sizes': np.array([k.size for k in kp]),
                    'angles': np.array([k.angle for k in kp]),
                    'responses': np.array([k.response for k in kp])
                }

                np.savez(os.path.join(self.output_path, f'features_{i}.npz'),
                        **kp_data, descriptors=des)

                features.append((kp, des))
                responses.append(kp_data['responses'])

        return features, responses

    def match_features(self, kp1, kp2, des1, des2):
        if des1 is None or des2 is None:
            return []

        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
        matches = bf.knnMatch(des1, des2, k=2)

        good_matches = []
        for m, n in matches:
            if m.distance < 0.65 * n.distance:
                good_matches.append(m)

        return good_matches if len(good_matches) >= 30 else []

    def reconstruct_3d(self, features):
        points_3d = []

        for i in range(len(features) - 1):
            matches = self.match_features(features[i][0], features[i+1][0],
                                       features[i][1], features[i+1][1])

            if matches:
                pts1 = np.float32([features[i][0][m.queryIdx].pt for m in matches])
                pts2 = np.float32([features[i+1][0][m.trainIdx].pt for m in matches])

                E, mask = cv2.findEssentialMat(pts1, pts2, self.K,
                                             method=cv2.RANSAC,
                                             prob=0.999,
                                             threshold=0.5)

                _, R, t, mask = cv2.recoverPose(E, pts1, pts2, self.K, mask=mask)

                P1 = self.K @ np.hstack((np.eye(3), np.zeros((3, 1))))
                P2 = self.K @ np.hstack((R, t))

                points_4d = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)
                points = points_4d[:3] / points_4d[3]
                points_3d.append(points.T)

        return np.vstack(points_3d) if points_3d else np.array([])

    def process_point_cloud(self, points_3d, responses):
        if len(points_3d) == 0:
            return None

        # Remove outliers
        clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
        mask = clf.fit_predict(points_3d) == 1
        points_3d = points_3d[mask]

        # Normalize coordinates
        min_coords = points_3d.min(axis=0)
        max_coords = points_3d.max(axis=0)
        points_3d = (points_3d - min_coords) / (max_coords - min_coords + 1e-6)

        # Create voxel grid
        grid_size = 1024
        grid = np.zeros((grid_size, grid_size, grid_size))
        voxel_coords = (points_3d * (grid_size - 1)).astype(int)

        # Compute density-based weights
        nbrs = NearestNeighbors(n_neighbors=10).fit(points_3d)
        distances, _ = nbrs.kneighbors(points_3d)
        density_weights = 1 / (np.mean(distances, axis=1) + 1e-6)

        # Apply weights to grid
        for point, weight in zip(voxel_coords, density_weights):
            grid[point[0], point[1], point[2]] += weight

        return gaussian_filter(grid, sigma=1.0)

    def visualize(self, points_3d, splatted_grid=None):
        fig = plt.figure(figsize=(15, 5))

        # Raw points visualization
        ax1 = fig.add_subplot(121, projection='3d')
        scatter1 = ax1.scatter(points_3d[:, 0], points_3d[:, 1], points_3d[:, 2],
                             c='blue', s=1, alpha=0.6)
        ax1.set_title("Raw 3D Points")
        ax1.view_init(elev=20, azim=45)

        # Gaussian splatted visualization
        if splatted_grid is not None:
            ax2 = fig.add_subplot(122, projection='3d')
            points = np.argwhere(splatted_grid > splatted_grid.mean() * 0.5)
            values = splatted_grid[points[:, 0], points[:, 1], points[:, 2]]
            sizes = 1 + (values - values.min()) * 10 / (values.max() - values.min())

            scatter2 = ax2.scatter(points[:, 0], points[:, 1], points[:, 2],
                                 c=values, cmap='viridis',
                                 s=sizes, alpha=0.6)
            ax2.set_title("Gaussian Splatted Points")
            ax2.view_init(elev=20, azim=45)
            plt.colorbar(scatter2)

        plt.tight_layout()
        plt.savefig("Gaussian_splatting_output.png")
        plt.show()

    def save_point_cloud(self, points_3d, file_name):
        if len(points_3d) == 0:
            print("No points to save.")
            return

        # Convert points to Open3D format
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points_3d)

        # Save as .ply file
        o3d.io.write_point_cloud(file_name, pcd)
        print(f"Point cloud saved to {file_name}")

def main():
    splatter = GaussianSplatting(data_path, output_features)

    try:
        images = splatter.load_images()
        features, responses = splatter.extract_features(images)
        points_3d = splatter.reconstruct_3d(features)

        if len(points_3d) > 0:
            splatted_grid = splatter.process_point_cloud(points_3d, responses)
            splatter.visualize(points_3d, splatted_grid)

            # Extract points from splatted grid (those with significant values)
            if splatted_grid is not None:
                splatted_points = np.argwhere(splatted_grid > splatted_grid.mean() * 0.5)
                splatted_points = splatted_points.astype(np.float32)

                # Save the Gaussian splatted 3D points as a .ply file
                output_ply_path = os.path.join(splatter.output_path, "gaussian_splatted_point_cloud.ply")
                splatter.save_point_cloud(splatted_points, output_ply_path)
            else:
                print("Splatted grid is empty. Unable to save splatted points.")
        else:
            print("Failed to reconstruct 3D points. Check input images and parameters.")

    except Exception as e:
        print(f"Error during processing: {str(e)}")

In [ ]:
main()